In [37]:
# !pip install nltk==3.6.5
# !pip install numpy==1.21.3
# !pip install pandas==1.3.3
# !pip install scikit-learn==0.24.2

In [27]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize



In [2]:

df = pd.read_csv('ipc_data.csv')



In [3]:

stop_words = set(stopwords.words('english'))



In [4]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]
    return ' '.join(filtered_tokens)



In [5]:
df=df[~df.Discription.isnull()]

In [6]:
df['Discription'] = df['Discription'].apply(preprocess_text)



/tmp/ipykernel_14686/807685686.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Discription'] = df['Discription'].apply(preprocess_text)


In [7]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Discription'])
y = df['Section']



In [8]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

classifier = MultinomialNB()
classifier.fit(X_train, y_train)



MultinomialNB()

In [9]:

y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)



Accuracy: 0.0


In [34]:

def recommend_sections(description, n=5):
    preprocessed_desc = preprocess_text(description)
    desc_vectorized = vectorizer.transform([preprocessed_desc])
    

    probas = classifier.predict_proba(desc_vectorized)
    
 
    top_n_indices = np.argsort(probas, axis=1)[:,-n:][:,::-1]

    top_n_sections = [classifier.classes_[idx] for idx in top_n_indices[0]]
    
    return top_n_sections

In [35]:

description = "Theft occurred in the store"
recommended_section = recommend_sections(description)
print("Recommended Section:", recommended_section)
description


Recommended Section: ['IPC Section 379', 'IPC Section 382', 'IPC Section 439', 'IPC Section 356', 'IPC Section 381']


'Theft occurred in the store'

In [36]:

description = "Murder"
recommended_section = recommend_sections(description)
print("Recommended Section:", recommended_section)


Recommended Section: ['IPC Section 303', 'IPC Section 396', 'IPC Section 307', 'IPC Section 165A', 'IPC Section 311']
